In [1]:
import ee
import os
PROJECT = 'pyregence-ee'
try:
    credentials = ee.ServiceAccountCredentials(email=None,key_file='/home/private-key.json')
    ee.Initialize(credentials)
except:
    ee.Initialize(PROJECT)

### Must choose source of baseline fuels data to update, pyrologix or firefactor

In [2]:
fuels_source = "pyrologix" # "firefactor"

### Make a folder to hold fuel layers generated from your treatment scenario ensemble

In [3]:
# for pc448 and others like it, provide paths to the DIST imgs representing the treatment scenarios you're updating Fuels for
scenario_paths = ["projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2"]

#create fuelscape folders for each DIST scenario
fuels_folders= [(path + "_fuelscape") for path in scenario_paths]

for fuels_folder in fuels_folders:
    fuels_folder_list = os.popen(f"earthengine ls {os.path.dirname(fuels_folder)}").read().split('\n')[0:-1]
    if not fuels_folder in fuels_folder_list:
        os.popen(f"earthengine create folder {fuels_folder}").read()
        print(f'Created Folder: {fuels_folder}')
    else:
        print(f"{fuels_folder} already exists")

Created Folder: projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape


### For each treated landscape img in your treatment scenarios imageCollection, create a new folder to hold the fuels layers

In [4]:
# don't need subfolder for this application
# scenarios_img_paths = os.popen(f"earthengine ls {scenario_paths}").read().split('\n')[0:-1]
# scn_sub_folders = [(fuels_folder+'/'+os.path.basename(path)) for path in scenarios_img_paths]

# sub_folders_list = os.popen(f"earthengine ls {fuels_folder}").read().split('\n')[0:-1]
# for sub_folder_name in scn_sub_folders:
#     if not sub_folder_name in sub_folders_list:
#         os.popen(f"earthengine create folder {sub_folder_name}").read()
#         print(f'Created scenario fuelscapes sub-folder: {sub_folder_name}')
#     else:
#         print(f"{sub_folder_name} already exists")

### Loop thru each fuel updater script the way we do for FireFactor: create_canopy_guide, calc_CC_CH, calc_CBD_CBH, calc_FM40 (don't need to run fm40 wui update)

In [4]:
# get config.yml file path, needed for every script
config_path = os.path.join(os.getcwd(),'config.yml')
print(config_path)

c:\pc448\config.yml


In [5]:
print(scenario_paths)
print(fuels_folders)

['projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2']
['projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape']


### Export Canopy Guide and FM40

In [6]:
# Canopy Guide
for scn_img_path,scn_sub_folder in zip(scenario_paths,fuels_folders):
    cmd = f"python src/CreateEEFuels/create_canopy_guide.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break

# FM40 
for scn_img_path,scn_sub_folder in zip(scenario_paths,fuels_folders):
    cmd = f"python src/CreateEEFuels/calc_FM40.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder} -f {fuels_source}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break


python src/CreateEEFuels/create_canopy_guide.py -c c:\pc448\config.yml -d projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2 -o projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape
python src/CreateEEFuels/calc_FM40.py -c c:\pc448\config.yml -d projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2 -o projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape -f pyrologix


### Export CC and CH (wait until all Canopy Guide exports have successfully completed before running)

In [7]:
# CC and CH
for scn_img_path,scn_sub_folder in zip(scenario_paths,fuels_folders):
    cmd = f"python src/CreateEEFuels/calc_CC_CH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder} -f {fuels_source}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    print('\n')
    #break

python src/CreateEEFuels/calc_CC_CH.py -c c:\pc448\config.yml -d projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2 -o projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape -f pyrologix




### Export CBD and CBH (wait until all CC and CH exports have completed before running)

In [8]:
# CBD and CBH
for scn_img_path,scn_sub_folder in zip(scenario_paths,fuels_folders):
    cmd = f"python src/CreateEEFuels/calc_CBD_CBH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder} -f {fuels_source}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    print('\n')
    #break

python src/CreateEEFuels/calc_CBD_CBH.py -c c:\pc448\config.yml -d projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2 -o projects/pyregence-ee/assets/pc448/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape -f pyrologix




### After all fuel layers in each scenario's fuelscape has successfully exported, collate each 5-layer fuelscape into a multiband img for Google Drive Export

In [9]:
from src.CreateEEFuels.utils.yml_params import get_export_params
AOI = ee.Image("projects/pyregence-ee/assets/pc448/templateImg").geometry()
for scn_img_path,scn_sub_folder in zip(scenario_paths,fuels_folders):
    # print(scn_img_path)
    # print(scn_sub_folder)
    fm40 = ee.ImageCollection(scn_sub_folder+'/fm40_collection').select('new_fbfm40').mosaic()
    cc = ee.Image(scn_sub_folder+'/CC')
    ch = ee.Image(scn_sub_folder+'/CH')
    cbh = ee.Image(scn_sub_folder+'/CBH')
    cbd = ee.Image(scn_sub_folder+'/CBD')
    fuel_stack = fm40.addBands(cc).addBands(ch).addBands(cbh).addBands(cbd).rename('FM40','CC','CH', 'CBH','CBD').toInt16()
    # fuel_stack = cc.addBands(ch).addBands(cbh).addBands(cbd).rename('CC','CH', 'CBH','CBD').toInt16() # exclude FM40
    # print(fuel_stack.bandNames().getInfo())

    # export
    scn_id = scn_sub_folder.split('/')[-1]
    # print(scn_id)
    
    desc = f"export_{scn_id}"
   
    crs,scale = get_export_params(config_path)
    # print(desc)
    # print(crs)
    # print(scale)
    folder=f'PC448_Fuelscapes'
    fileNamePrefix=scn_id
    
    task = ee.batch.Export.image.toDrive(image=fuel_stack,description=desc,folder=folder,fileNamePrefix=fileNamePrefix,region=AOI,scale=scale,crs=crs)
    task.start()
    print(f'Export started: {folder}/{fileNamePrefix}')
    #break

Export started: PC448_Fuelscapes/DIST_Treatments_Plumas_Protect_Alt1_v2_fuelscape
